In [1]:
import cv2
import numpy as np
%matplotlib inline


In [2]:
image = '/home/donghohan/compvision/Computer-Vision-based-Offside-Detection-in-Soccer/dataset/Offside_Images/6.jpg'
test_result = './'
side = 'right'

In [5]:
# selectedLines = get_vertical_lines(img , side)
img = cv2.imread(image)
selectedLines = []
selectedLinesParams = []
linesFound = False
BlueRedMask = 100

In [14]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv, (35, BlueRedMask, BlueRedMask), (70, 255,255))
imask = mask>0
cv2.imwrite("HSV.png", hsv)
cv2.imwrite("mask.png", mask)



True

In [18]:
green = np.zeros_like(img, np.uint8)
green[imask] = img[imask]
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.imwrite("green.png", green)
cv2.imwrite("gray.png", gray)
edges = cv2.Canny(green,150,250,apertureSize = 3)
lines = cv2.HoughLines(edges,1,np.pi/180, 200) 
cv2.imwrite("edges.png", edges)

True

In [ ]:
def det(a, b):
    return a[0] * b[1] - a[1] * b[0]

def line_intersection(line1, line2):
    x_diff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    y_diff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    div = det(x_diff, y_diff)
    if div == 0:
        return None

    d = (det(*line1), det(*line2))
    x = det(d, x_diff) / div
    y = det(d, y_diff) / div

    return [x, y]

def find_intersections(lines):
    intersections = []
    for i, line_1 in enumerate(lines):
        for line_2 in lines[i + 1:]:
            if not line_1 == line_2:
                intersection = line_intersection(line_1, line_2)
                if intersection:
                    intersections.append(intersection)

    return intersections

In [ ]:
def get_vertical_lines(image , side):
    img = image
    selectedLines = []
    selectedLinesParams = []
    linesFound = False
    BlueRedMask = 100
    
    
    
    
    while linesFound == False:
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, (35, BlueRedMask, BlueRedMask), (70, 255,255))
        # boolean ndarray
        imask = mask>0
        green = np.zeros_like(img, np.uint8)
        green[imask] = img[imask]
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # v = np.median(green)
        # sigma = 0.33
        # lower = int(max(0, (1.0 - sigma) * v))
        # upper = int(min(255, (1.0 + sigma) * v))
        cv2.imwrite('green.jpg', green)
        edges = cv2.Canny(green,150,250,apertureSize = 3) 
        # minLineLength = 1
        # maxLineGap = 1250
        lines = cv2.HoughLines(edges,1,np.pi/180, 200)
        if lines.any():
            if len(lines) > 2:  
                linesFound = True  
            else: 
                BlueRedMask -= 10

    
    
    linesFound = False
    
    if side == 'left':
    	angleMaxLimit = 20
    	angleMinLimit = 70
    else:
    	angleMaxLimit = 150
    	angleMinLimit = 105
    
    rLimit = 300
    while linesFound == False:
        for line in lines: 
            for r,theta in line:
                isLineValid = True
                a = np.cos(theta) 
                b = np.sin(theta)
                
                if (theta * 180 * 7 / 22) > angleMinLimit and (theta * 180 * 7 / 22) < angleMaxLimit:
                    #print(theta * 180 * 7 / 22 , r)
                    x0 = a*r 
                    y0 = b*r 
                    x1 = int(x0 + 1000*(-b)) 
                    y1 = int(y0 + 1000*(a)) 
                    x2 = int(x0 - 1000*(-b)) 
                    y2 = int(y0 - 1000*(a))
                    
                    #cv2.line(image,(x1,y1), (x2,y2), (0,0,255),1)
                    
                    
                    if len(selectedLines) > 0: 
                        for lineParams in selectedLinesParams:
                            #print(abs(lineParams[0] - r))
                            if abs(lineParams[0] - r) < rLimit:
                                isLineValid = False
                        for selectedLine in selectedLines:
                            if not line_intersection(selectedLine, [[x1,y1],[x2,y2]]):
                                isLineValid = False
                        if [[x1,y1],[x2,y2]] in selectedLines or [[x2,y2],[x1,y1]] in selectedLines:
                            isLineValid = False
                    if isLineValid:
                        #print("P")
                        selectedLines.append([[x1,y1],[x2,y2]])
                        print(x1,y1,x2,y2)
                        selectedLinesParams.append([r, theta])
                        cv2.line(image,(x1,y1), (x2,y2), (0,0,255),1)
                        cv2.putText(image, str((theta * 180 * 7 / 22)) ,(int((x2))  ,  int((y2))) , cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
                        cv2.putText(image, str((theta * 180 * 7 / 22)) ,(int((x1))  ,  int((y1))) , cv2.FONT_HERSHEY_SIMPLEX, 1, (200,255,155), 2, cv2.LINE_AA)
        if len(selectedLines) < 2:
            if rLimit >= 75:
                rLimit -= 10
            else:
                angleMinLimit -= 1
                angleMaxLimit += 1
                rlimit = 100
        else:
            #for par in selectedLinesParams:
            #	print( "Angle", (par[1]*180*7)/22)
            linesFound = True
        
    return selectedLines


In [ ]:
def get_vertical_vanishing_point(img , side):
    
    selectedLines = get_vertical_lines(img , side)
    intersectionPoints = find_intersections(selectedLines)
    vanishingPointX = 0.0
    vanishingPointY = 0.0
    for point in intersectionPoints:
        vanishingPointX += point[0]
        vanishingPointY += point[1]
    # print(selectedLines)

    return (vanishingPointX/len(intersectionPoints), vanishingPointY/len(intersectionPoints))